In [10]:
import pandas as pd
import glob
import numpy as np
import _pickle as cPickle
import warnings
warnings.filterwarnings("ignore")
import os
print(os.getcwd())

/home/liangzhp/dp-SCO-streaming-code


In [11]:
def paras(x):
    return x.split('=')[-1]

dfbandit = pd.DataFrame(columns = ['T', 'd', 'algo', 'scale'])
dfsco = pd.DataFrame(columns = ['T', 'd', 'p', 'algo', 'scale', 'noise_free', 'test_flag'])
repo_paths = ['dpsco-results']
for repo_path in repo_paths:
    root_path = '{}/*/*.pkl'.format(repo_path)
    for item in glob.glob(root_path):
        setting, algo = item.split('/')[-2], item.split('/')[-1]
        p, d, T, scale, noise_free, test_flag = setting.split('-')
        if p=='p=1.0':
            with open(item, "rb")  as input_file:
                dict_list = cPickle.load(input_file)
                est_mean = np.mean([logger_dict['result']['record'][-1][1] for logger_dict in dict_list])
                est_std = np.std([logger_dict['result']['record'][-1][1] for logger_dict in dict_list])
                regret_mean = np.mean([np.sum(record[2] for record in logger_dict['result']['record']) for logger_dict in dict_list])
                regret_std = np.std([np.sum(record[2] for record in logger_dict['result']['record']) for logger_dict in dict_list])
                time_mean = np.mean([logger_dict['result']['record'][-1][3] for logger_dict in dict_list])
                time_std = np.std([logger_dict['result']['record'][-1][3] for logger_dict in dict_list])
            dfbandit = dfbandit.append(
                {"T": paras(T),
                "d": paras(d),
                'algo': algo[:-4],
                "scale": paras(scale),
                "est_mean": est_mean,
                "est_std": est_std,
                "regret_mean": regret_mean,
                "regret_std": regret_std,
                "time_mean": time_mean,
                "time_std": time_std
                }, ignore_index=True)
        else:
            with open(item, "rb")  as input_file:
                dict_list = cPickle.load(input_file)
                if dict_list[0]['algo']['test_flag']==True:
                    risk_mean = np.mean([logger_dict['result']['record'][-1][1] for logger_dict in dict_list])
                    risk_std = np.std([logger_dict['result']['record'][-1][1] for logger_dict in dict_list])
                    subopt_mean = np.mean([logger_dict['result']['record'][-1][2] for logger_dict in dict_list])
                    subopt_std = np.std([logger_dict['result']['record'][-1][2] for logger_dict in dict_list])
                    time_mean, time_std = -1, -1
                else:
                    time_mean = np.mean([logger_dict['result']['time'][-1] for logger_dict in dict_list])
                    time_std = np.std([logger_dict['result']['time'][-1] for logger_dict in dict_list])
                    risk_mean, risk_std, subopt_mean, subopt_std = -1, -1, -1, -1

                dfsco = dfsco.append(
                {"T": paras(T),
                "d": paras(d),
                "p": paras(p),
                'algo': algo[:-4],
                "scale": paras(scale),
                "noise_free": paras(noise_free),
                "test_flag": paras(test_flag),
                "risk_mean": risk_mean,
                "subopt_mean": subopt_mean,
                "time_mean": time_mean,
                "risk_std": risk_std,
                "subopt_std": subopt_std,
                "time_std": time_std
                }, ignore_index=True)
dfbandit = dfbandit.replace(-1, 10000)
dfsco = dfsco.replace(-1, 10000)
dfsco = dfsco.replace({"OFW_ple2":"OFW", "OFW_pge2":"OFW", "Local_MD":"LocalMD"})
dfsco_time = dfsco[dfsco['test_flag'] == 'False']
dfsco_risksub  = dfsco[dfsco['test_flag'] == 'True']

dpsco-results/p=1.5-d=5-T=2000-scale=0.6-noise_free=False-test_flag=False/OFW_ple2.pkl
dpsco-results/p=inf-d=50-T=10000-scale=0.8-noise_free=False-test_flag=True/OFW_pge2.pkl
dpsco-results/p=inf-d=50-T=2000-scale=0.6-noise_free=False-test_flag=True/OFW_pge2.pkl
dpsco-results/p=1.5-d=5-T=5000-scale=0.1-noise_free=True-test_flag=False/OFW_ple2.pkl
dpsco-results/p=1.5-d=50-T=2000-scale=0.6-noise_free=False-test_flag=True/OFW_ple2.pkl
dpsco-results/p=inf-d=50-T=10000-scale=0.7-noise_free=False-test_flag=True/OFW_pge2.pkl
dpsco-results/p=1.5-d=10-T=10000-scale=0.2-noise_free=True-test_flag=True/OFW_ple2.pkl
dpsco-results/p=1.5-d=50-T=2000-scale=0.2-noise_free=True-test_flag=True/OFW_ple2.pkl
dpsco-results/p=1.5-d=5-T=1000-scale=0.4-noise_free=False-test_flag=False/OFW_ple2.pkl
dpsco-results/p=inf-d=5-T=1000-scale=0.7-noise_free=False-test_flag=True/OFW_pge2.pkl
dpsco-results/p=1.5-d=50-T=1000-scale=0.6-noise_free=False-test_flag=False/OFW_ple2.pkl
dpsco-results/p=inf-d=10-T=1000-scale=0.8-n

In [14]:
def analyze_sco(df_time, df_risksub):
    # df_groupby = df.groupby(by=['T','d','p', 'algo', 'noise_free', 'scale', 'test_flag']).agg({'risk_mean': ['min'], 'subopt_mean': 'min', 'time_mean': 'min'})
    # df_groupby_testfalse = df_groupby.loc[(slice(None), slice(None), slice(None), slice(None), ["False"], slice(None), ['True']), :].droplevel(["noise_free"])
    df_risksub = df_risksub.sort_values('risk_mean', ascending=True).drop_duplicates(['T','d','p', 'algo', 'noise_free'])
    df_risksub = df_risksub[df_risksub['noise_free']=='False']
    df_time = df_time[df_time['noise_free'] == 'False']
    df_summary = df_risksub[['T','d','p', 'algo', 'scale', 'risk_mean', 'risk_std', 'subopt_mean', 'subopt_std']].merge(df_time[['T','d','p', 'algo', 'scale', 'time_mean', 'time_std']], left_on=['T','d','p', 'algo', 'scale'], right_on=['T','d','p', 'algo', 'scale'], how = 'left')
    df_temp = df_summary.groupby(by = ['T','d','p', 'algo']).mean().loc[(["1000", "2000", "5000", "10000"], ["5", "10", "20"], slice(None), slice(None))]
    df_temp[['risk_mean', 'risk_std', 'subopt_mean', 'subopt_std', 'time_mean', 'time_std']] = df_temp[['risk_mean', 'risk_std', 'subopt_mean', 'subopt_std', 'time_mean', 'time_std']].applymap('{:,.3g}'.format)
    df_temp_str = df_temp[['risk_mean', 'risk_std', 'subopt_mean', 'subopt_std', 'time_mean', 'time_std']].astype(str)
    df_temp_str['link'] = '$\pm$'
    df_temp_str['Risk'] = df_temp_str['risk_mean'] + df_temp_str['link'] + df_temp_str['risk_std']
    df_temp_str['SubOpt'] = df_temp_str['subopt_mean'] + df_temp_str['link'] + df_temp_str['subopt_std']
    df_temp_str['Time'] = df_temp_str['time_mean'] + df_temp_str['link'] + df_temp_str['time_std']
    # df_temp_str.loc[(slice(None), slice(None), slice(None), 'LocalMD')] = '-'
    return df_temp_str

df_temp_str  = analyze_sco(dfsco_time, dfsco_risksub)
# df_temp_str[['Risk', 'SubOpt', 'Time']].to_latex(buf = '../preprint/entries/exp_table.tex', multirow=True, escape=False, label = 'tab:exp', caption = 'Experiment Results')
df_temp_str[['Risk', 'SubOpt', 'Time']]

Risk                 SubOpt  \
T     d  p   algo                                                
1000  5  1.5 OFW   0.00353$\pm$0.000474    0.00597$\pm$0.00278   
         inf OFW      0.0272$\pm$0.0153      0.0834$\pm$0.0521   
      10 1.5 OFW     0.0154$\pm$0.00332         0.132$\pm$0.03   
         inf OFW      0.0479$\pm$0.0175        0.598$\pm$0.215   
      20 1.5 OFW     0.0439$\pm$0.00445       0.697$\pm$0.0612   
         inf OFW      0.0318$\pm$0.0031         1.53$\pm$0.152   
2000  5  1.5 OFW   0.00285$\pm$0.000243     0.0023$\pm$0.00154   
         inf OFW     0.0139$\pm$0.00387      0.0394$\pm$0.0135   
      10 1.5 OFW    0.00769$\pm$0.00148      0.0546$\pm$0.0158   
         inf OFW      0.0364$\pm$0.0189         0.45$\pm$0.243   
      20 1.5 OFW     0.0272$\pm$0.00601         0.4$\pm$0.0878   
         inf OFW     0.0278$\pm$0.00373         1.26$\pm$0.154   
5000  5  1.5 OFW   0.00263$\pm$0.000153  0.000887$\pm$0.000461   
         inf OFW    0.00744$\pm$0.00172     0.0167$\pm$0.00537   
      10 1.5 OFW   0.00408$\pm$0.000398     0.0163$\pm$0.00387   
         inf OFW     0.0177$\pm$0.00212       0.199$\pm$0.0299   
      20 1.5 OFW     0.0104$\pm$0.00227       0.135$\pm$0.0427   
         inf OFW     0.0198$\pm$0.00573        0.915$\pm$0.323   
10000 5  1.5 OFW   0.00251$\pm$3.98e-05  0.000233$\pm$0.000111   
         inf OFW    0.0035$\pm$0.000452     0.00353$\pm$0.0015   
      10 1.5 OFW   0.00294$\pm$0.000163   0.00431$\pm$0.000736   
         inf OFW    0.00727$\pm$0.00119       0.062$\pm$0.0156   
      20 1.5 OFW   0.00524$\pm$0.000717      0.0465$\pm$0.0104   
         inf OFW      0.0124$\pm$0.0034        0.523$\pm$0.168   

                                 Time  
T     d  p   algo                      
1000  5  1.5 OFW     0.105$\pm$0.0519  
         inf OFW     0.073$\pm$0.0159  
      10 1.5 OFW     0.123$\pm$0.0609  
         inf OFW   0.0647$\pm$0.00787  
      20 1.5 OFW     0.116$\pm$0.0419  
         inf OFW    0.0781$\pm$0.0288  
2000  5  1.5 OFW     0.163$\pm$0.0403  
         inf OFW      0.144$\pm$0.028  
      10 1.5 OFW     0.186$\pm$0.0312  
         inf OFW     0.156$\pm$0.0365  
      20 1.5 OFW     0.176$\pm$0.0357  
         inf OFW     0.137$\pm$0.0166  
5000  5  1.5 OFW      0.437$\pm$0.117  
         inf OFW     0.289$\pm$0.0283  
      10 1.5 OFW      0.473$\pm$0.142  
         inf OFW       0.3$\pm$0.0615  
      20 1.5 OFW      0.462$\pm$0.108  
         inf OFW     0.315$\pm$0.0604  
10000 5  1.5 OFW      0.859$\pm$0.143  
         inf OFW      0.663$\pm$0.112  
      10 1.5 OFW       0.92$\pm$0.243  
         inf OFW     0.573$\pm$0.0368  
      20 1.5 OFW      0.903$\pm$0.146  
         inf OFW       0.564$\pm$0.13

In [13]:
dfbandit = dfbandit.sort_values('regret_mean', ascending=True).drop_duplicates(['T','d','algo'])
df_temp = dfbandit.groupby(by = ['T','d', 'algo']).mean().loc[(["1000", "2000", "5000", "10000"], ["10", "20", "50"], slice(None), slice(None))]
feature_subset = ['regret_mean', 'regret_std', 'time_mean', 'time_std']
df_temp[feature_subset] = df_temp[feature_subset].applymap('{:,.3g}'.format)
df_temp_str = df_temp[feature_subset].astype(str)
df_temp_str['link'] = '$\pm$'
df_temp_str['Regret'] = df_temp_str['regret_mean'] + df_temp_str['link'] + df_temp_str['regret_std']
df_temp_str['Time'] = df_temp_str['time_mean'] + df_temp_str['link'] + df_temp_str['time_std']
df_temp_str[['Regret', 'Time']]

KeyError: 'regret_mean'